In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col
from time import time
import sys

In [0]:
sys.path.append("/Workspace/Users/faraaz_khan@epam.com/PEI/utils")

In [0]:
from common_util import normalize_columns

In [0]:
start_time = time()

In [0]:
source_path = "/Volumes/fz_catalog/landing/products/"
checkpoint_path = "/Volumes/fz_catalog/landing/_checkpoints/products_bronze/"

In [0]:
# Step 3: Read with explicit schema
df = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("quote", '"')
    .option("escape", '"')
    .option("cloudFiles.schemaLocation", f"{checkpoint_path}")
    .option("cloudFiles.schemaEvolutionMode", "rescue")
    .load(source_path)
)

df_renamed = normalize_columns(df)


df_with_audit = df_renamed.withColumn("input_file_name", col("_metadata.file_name")).withColumn(
    "file_modification_time", col("_metadata.file_modification_time")
)



df_with_audit.writeStream.format("delta")\
.outputMode("append")\
.option("checkpointLocation", checkpoint_path)\
.trigger(availableNow=True)\
.toTable("fz_catalog.bronze.products")

In [0]:
end_time = time()

In [0]:
dbutils.notebook.exit(f"Run Time: {end_time - start_time} seconds")

I am considering that schema can be changes here and if any ne column comes, it will be stored in rescure columns